In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pwd

/content


In [2]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/project 2 updated')

In [3]:
import os
import numpy as np
# from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
# from datetime import datetime
# import cv2
# from PIL import Image
# from keras import backend, optimizers

In [4]:
image_directory = 'Att_UNet Data/Training_set/image/'
mask_directory = 'Att_UNet Data/Training_set/mask/'

#SIZE = 256
image_dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.
mask_dataset = []  #Place holders to define add labels. We will add 0 to all parasitized images and 1 to uninfected.

images = os.listdir(image_directory)
masks = os.listdir(mask_directory)


In [ ]:
#train_data = np.array([])
#mask_data = np.array([])

In [5]:
for img in images:
    train_img = np.load('Att_UNet Data/Training_set/image/' + str(img))
    #np.append(train_data, train_img)
    image_dataset.append(train_img)
    print(img)

for msk in masks:
    train_msk = np.load('Att_UNet Data/Training_set/mask/' + str(msk))
    train_msk_reshaped = train_msk.reshape((128, 128, 128, 4))
    #np.append(mask_data, train_msk_reshaped)
    mask_dataset.append(train_msk_reshaped)
    print(msk)

image_0.npy
image_1.npy
image_2.npy
image_3.npy
image_4.npy
image_7.npy
mask_0.npy
mask_1.npy
mask_2.npy
mask_3.npy
mask_4.npy
mask_7.npy


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.10, random_state = 0)
#X_train = np.array(X_train)
print(X_train[0].shape)
#y_train = np.array(y_train)

(128, 128, 128, 3)


In [7]:
IMG_HEIGHT = X_train[1].shape[0]
IMG_WIDTH  = X_train[1].shape[1]
IMG_DEPTH  = X_train[1].shape[2]
IMG_CHANNELS = X_train[1].shape[3]
num_labels = 4  #Binary
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH, IMG_CHANNELS)
batch_size = 1

In [ ]:
!pip install segmentation_models_3D

In [8]:
import segmentation_models_3D as sm
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.25,0.25,0.25,0.25]))
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

Segmentation Models: using `tf.keras` framework.


In [9]:
from Attention_U_Net import Attention_UNet, dice_coef, dice_coef_loss, jacard_coef

att_unet_model = Attention_UNet(input_shape)

att_unet_model.compile(optimizer=Adam(lr = 1e-2), loss=total_loss,
              metrics=['accuracy', jacard_coef])

X_train2 = np.stack(X_train)
tra = X_train[0][np.newaxis,:,:,:,:]
y_train2 = np.stack(y_train)
lab = y_train[0][np.newaxis,:,:,:,:]
X_test2 = np.stack(X_test)
y_test2 = np.stack(y_test)
print(X_train2.shape,y_train2.shape)
print(type(lab), type(tra))

Input shape: (128, 128, 128, 3)
Input shape: (128, 128, 128, 3)


/usr/local/lib/python3.10/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


(5, 128, 128, 128, 3) (5, 128, 128, 128, 4)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:
att_unet_model.summary()

In [ ]:
#tf.config.run_functions_eagerly(True)

In [ ]:
att_unet_history = att_unet_model.fit(X_train2, y_train2,
                    verbose=1,
                    validation_split = 0.15,
                    batch_size = batch_size,
                    validation_data=[X_test2, y_test2],
                    shuffle=False,
                    epochs=3)

Epoch 1/3


In [10]:
print(X_train2.shape,y_train2.shape)
print(X_test2.shape,y_test2.shape)

(5, 128, 128, 128, 3) (5, 128, 128, 128, 4)
(1, 128, 128, 128, 3) (1, 128, 128, 128, 4)
